In [ ]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import numpy as np
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    N_REDUCED_SCNEARIOS,
    ALL_VEHICLE_TYPES,
    PATH_RESULTS_SUMMARY,
    PATH_RESULTS_SINGLE_MODAL_BENCHMARK,
    PATH_RESULTS_VAR_REGION,
    PATH_RESULTS_VAR_TUPLE,
    PATH_RESULTS_VALUE_STOCHASTIC,
    PATH_FLEET_SIZE,
    PATH_SCENARIO_TREE_NODES,
    PATH_RESULTS_VALUE_AT_RISK,
)
from modules.stochastic_program.factory import StochasticProgramFactory, StochasticProgram



# Stochastic Program
In this notebook we will use the previously prepared data to create a stochastic program and solve it. In order to evaluate its performance we will perform multiple benchmarks.
## Read Input Data

In [ ]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [ ]:
node_df = pd.read_pickle(PATH_SCENARIO_TREE_NODES)

In [ ]:
distances = pd.read_pickle(PATH_DISTANCES).round(2)

In [ ]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [ ]:
real_fleet_size = pd.read_pickle(PATH_FLEET_SIZE).to_dict()["id"]
real_fleet_size = {
    vehicle_type: int(fleet_size/3) for vehicle_type, fleet_size in real_fleet_size.items()
}
real_fleet_size

{'bicycle': 996, 'car': 658, 'kick_scooter': 2600}

# Create & Solve Stochastic Program

To create and solve the stochastic program we use the `StochasticProgramFactory` and `StochasticProgram` classes from our own module.  
We first create the stochastic program factory with all of the prepared data.

In [ ]:
vehicle_types = ALL_VEHICLE_TYPES
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    vehicle_types,
)
factory.set_initial_allocation(real_fleet_size)


_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.01 seconds
_convert_demand finished in 0.13 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.14 seconds
_set_max_demand finished in 0.01 seconds
set_initial_allocation finished in 0.01 seconds


Now we create the stochastic program using the factory. The factory will pass all the necessary data in the correct format to the stochastic program class.

In [ ]:
stochastic_program: StochasticProgram = factory.create_stochastic_program()

create_stochastic_program finished in 0.00 seconds


We now use the stochastic program class to create the constraints and the objective function using the pulp library.  
We also can configure the model to be risk averse by setting a beta larger than zero. Beta is the weight of the calculate-value-at-risk. Look at the documentation in `stochastic_program.py` for further details.

In [ ]:
stochastic_program.create_model(beta=0.0, alpha=0.75)

_create_variables finished in 0.28 seconds
_create_objectives finished in 0.38 seconds
_create_demand_constraints finished in 0.48 seconds
_create_idle_vehicle_binary_constraints finished in 0.01 seconds
_create_big_u_sum_constraints finished in 0.06 seconds
_create_unfulfilled_demand_binary_constraints finished in 0.01 seconds
_create_no_refused_demand_constraints finished in 0.01 seconds
_create_relocations_constraints finished in 0.14 seconds
_create_vehicle_trips_starting_constraints finished in 0.09 seconds
_create_vehicle_trips_ending_constraints finished in 0.08 seconds
_create_initial_allocation_constraints finished in 0.00 seconds
_create_non_anticipativity_constraints finished in 0.26 seconds
_create_constraints finished in 1.14 seconds
create_model finished in 2.11 seconds


We now can solve the formulated program. Make sure that the solver selected in `config.py` is configured properly.

In [ ]:
print(stochastic_program._get_solver())
stochastic_program.solve()

No parameters matching '_test' found
No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  23305.031111109853
solve finished in 847.24 seconds


Here we can take a look at the dimensions of the problem.

In [ ]:
scenarios.reset_index().nunique()

scenarios         4
start_hex_ids    29
end_hex_ids      29
time              3
vehicle_types     3
demand           85
dtype: int64

In [ ]:
stochastic_program.get_summary()

get_results_by_tuple_df finished in 0.22 seconds
get_results_by_region_df finished in 0.01 seconds
get_summary finished in 0.30 seconds


{'status': 'Optimal',
 'objective': 23305.031111109853,
 'expected_profit': 23305.031111109853,
 'n_trips_avg': 8026.75,
 'n_unfilled_demand_avg': 1627.5,
 'demand_avg': 9654.25,
 'n_parking_avg': 4688.0,
 'n_relocations_avg': 2367.0}

In [ ]:
results_by_region = stochastic_program.get_results_by_region_df()
results_by_tuple = stochastic_program.get_results_by_tuple_df()

os.makedirs(os.path.dirname(PATH_RESULTS_VAR_REGION), exist_ok=True)
results_by_region.to_pickle(PATH_RESULTS_VAR_REGION)

os.makedirs(os.path.dirname(PATH_RESULTS_VAR_TUPLE), exist_ok=True)
results_by_tuple.to_pickle(PATH_RESULTS_VAR_TUPLE)

get_results_by_region_df finished in 0.01 seconds
get_results_by_tuple_df finished in 0.21 seconds


# Benchmarks
We will now run different benchmarks on our model to evaluate its performance.
## Different Capacities/ Disabled Relocations/ Value Of Perfect Information
In our first benchmark we will solve the model with 3 different vehicle fleet sizes to see how the fleet size affects the performance of the model.  
For each fleet size we will also enable/disable relocations, so that we can see how relocations can improve profit and demand fulfillment. We will also enable/disable the non-anticipativty constraints to calculate the value of perfect information.

In [ ]:
capacities = [
    {vehicle_type: round(size * 0.75) for vehicle_type, size in real_fleet_size.items()},
    {vehicle_type: round(size * 1) for vehicle_type, size in real_fleet_size.items()},
    {vehicle_type: round(size * 1.25) for vehicle_type, size in real_fleet_size.items()},
]
capacities

[{'bicycle': 747, 'car': 494, 'kick_scooter': 1950},
 {'bicycle': 996, 'car': 658, 'kick_scooter': 2600},
 {'bicycle': 1245, 'car': 822, 'kick_scooter': 3250}]

In [ ]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)

factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.01 seconds
_convert_demand finished in 0.03 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.04 seconds
_set_max_demand finished in 0.01 seconds
No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  23982.74999999992
solve finished in 11.02 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  24473.041111111062
solve finished in 10.13 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  7389.98111111112
solve finished in 2.89 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  7476.041111111163
solve finished in 1.81 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  23305.031111109853
solve finished in 845.67 seconds


No parameters matching '_t

In [ ]:
results_df = pd.DataFrame.from_dict(results)

In [ ]:
results_df

,status,objective,expected_profit,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,bicycle,car,kick_scooter,relocations_disabled,non_anticipativity_disabled
0,Optimal,23982.750000,23982.750000,7472.50,2181.75,9654.25,2081.25,2123.00,747,494,1950,False,False
1,Optimal,24473.041111,24473.041111,7507.50,2146.75,9654.25,2064.75,2223.25,747,494,1950,False,True
2,Optimal,7389.981111,7389.981111,4090.00,5564.25,9654.25,5458.25,0.00,747,494,1950,True,False
3,Optimal,7476.041111,7476.041111,4089.00,5565.25,9654.25,5484.00,0.00,747,494,1950,True,True
4,Optimal,23305.031111,23305.031111,8026.75,1627.50,9654.25,4688.00,2367.00,996,658,2600,False,False
5,Optimal,24083.468889,24083.468889,8392.75,1261.50,9654.25,4363.75,2413.25,996,658,2600,False,True
6,Optimal,4730.858889,4730.858889,5123.50,4530.75,9654.25,7638.50,0.00,996,658,2600,True,False
7,Optimal,4783.520000,4783.520000,5130.50,4523.75,9654.25,7631.50,0.00,996,658,2600,True,True
8,Optimal,20413.967778,20413.967778,8193.75,1460.50,9654.25,7756.75,2147.00,1245,822,3250,False,False
9,Optimal,21360.241111,21360.241111,8541.50,1112.75,9654.25,7400.25,2316.75,1245,822,3250,False,True


In [ ]:
os.makedirs(os.path.dirname(PATH_RESULTS_SUMMARY), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SUMMARY)


## Single Modal Benchmark
The novelty of our model is the consideration of multiple vehicle types. With this benchmark we will examine whether this consideration actually improves the profit. To do that we will create three single modal models and run them subsequently. The first single modal model will only consider kick scooters. We will then extract the unfulfilled demand from the solution of the first model and use that as an additional input for the second single modal model, which considers bicycles. Therefore the second model will have the bicycle demand plus the unfulfilled demand from the kick scooters as input. We will then repeat the process for cars.  
We can then compare the sum of the profit of all three single modal models with the profit of the multi modal model. 

In [ ]:
scenarios_copy = scenarios.copy().reset_index()

demand_per_type = {
    "car": scenarios_copy[scenarios_copy['vehicle_types'] == 'car']\
        .set_index(['scenarios','start_hex_ids','end_hex_ids','time' ,'vehicle_types']),
    "kick_scooter": scenarios_copy[scenarios_copy['vehicle_types'] == 'kick_scooter']\
        .set_index(['scenarios','start_hex_ids','end_hex_ids','time' ,'vehicle_types']),
    "bicycle": scenarios_copy[scenarios_copy['vehicle_types'] == 'bicycle']\
        .set_index(['scenarios','start_hex_ids','end_hex_ids','time' ,'vehicle_types']),
}

In [ ]:
results = []
# last demand in first iteration is 0
last_demand = pd.DataFrame(index=demand_per_type['car'].index)
last_demand['demand'] = 0
for vehicle_types in [["kick_scooter"], ["bicycle"], ["car"]]:
    current_vehicle_type = vehicle_types[0]
    print("previous demand", demand_per_type[current_vehicle_type]['demand'].values.sum(), "\n")
    print("demand for current vehicle", last_demand['demand'].values.sum(), "\n")
    current_demand = demand_per_type[current_vehicle_type] \
                    .reset_index('vehicle_types') \
                    .drop("vehicle_types", axis=1) \
                    + last_demand.reset_index('vehicle_types').drop("vehicle_types", axis=1)
    current_demand['vehicle_types'] = current_vehicle_type
    current_demand = current_demand.set_index('vehicle_types', append=True)
    print("total current demand", current_demand['demand'].values.sum(), "\n")
    current_fleet_capacity = {
        current_vehicle_type: real_fleet_size[current_vehicle_type]
    }

    current_demand.index = current_demand.index.set_levels(
        current_demand.index.get_level_values("vehicle_types").map(
            lambda x: current_vehicle_type
        ),
        verify_integrity=False,
        level="vehicle_types",
    )

    factory = StochasticProgramFactory(
        current_demand,
        distances,
        probabilities,
        node_df,
        vehicle_types,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    stochastic_program.create_model()
    stochastic_program.solve()
    # we transform the unfulfilled demand of the current lp into the demand for the next lp
    last_demand = stochastic_program.get_unfulfilled_demand().rename(
        columns={"accumulated_unfulfilled_demand": "demand"}
    )
    
    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": str(vehicle_types)}
    )


previous demand 24376 

demand for current vehicle 0 

total current demand 24376 

Academic license - for non-commercial use only - expires 2021-07-23
Using license file /home/moritz/licenses/gurobi.lic
No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  10314.058888888907
solve finished in 0.59 seconds
previous demand 12915 

demand for current vehicle 2885 

total current demand 15800 

No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  6734.591111111116
solve finished in 0.56 seconds
previous demand 1326 

demand for current vehicle 6556 

total current demand 7882 

No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  -13454.215555555567
solve finished in 0.57 seconds


In [ ]:
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    ALL_VEHICLE_TYPES,
    include_methods=[None],
)
factory.set_initial_allocation(real_fleet_size)
stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]
stochastic_program.create_model()
stochastic_program.solve()

results.append(
    {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
)


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  23305.031111109853
solve finished in 869.70 seconds


In [ ]:
results = pd.DataFrame.from_dict(results)

In [ ]:
# as unfulfilled demand of bicycles and kick scooters is transferred, we should not count that unfulfilled demand when summing up all unfulfilled demand
results.loc[[0,1], 'n_unfilled_demand_avg'] = 0

In [ ]:
compare = results.iloc[[3]]
compare = compare.append(results.iloc[range(3)].sum(), ignore_index=True)
# demand avg cannot be interpreted as some demand would be counted twice or more
compare = compare.drop(columns=['demand_avg'])
compare

\begin{tabular}{lrrrr}
\toprule
{} &     objective &  n\_trips\_avg &  n\_unfilled\_demand\_avg &  n\_relocations\_avg \\
\midrule
0 &  23305.031111 &      8026.75 &                1627.50 &             2367.0 \\
1 &   3594.434444 &      8735.50 &                 918.75 &             2919.0 \\
\bottomrule
\end{tabular}



In [ ]:
os.makedirs(os.path.dirname(PATH_RESULTS_SINGLE_MODAL_BENCHMARK), exist_ok=True)
compare.to_pickle(PATH_RESULTS_SINGLE_MODAL_BENCHMARK)

## Value Of The Stochastic Solution
We tried to calculate the value of the stochastic solution. However, this we could not simply use the average demand values from all scenarios, as those can be non-integer. Therefore we rounded non-integer values up and down and then solved with both the floored and the ceiled values. As this method does not lead to the actual value of the stochastic solution, we did not include it in our paper.  
The following results should be treated with caution.

In [ ]:
demand = scenarios.copy()
demand = (
    demand.unstack("scenarios")["demand"]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0: "demand"})
)
demand["scenarios"] = 0
demand = demand.set_index("scenarios", append=True).reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)
demand['demand'] = demand['demand'] / N_REDUCED_SCNEARIOS
demand['floored'] = demand['demand'].apply(np.floor)
demand['ceiled'] = demand['demand'].apply(np.ceil)


In [ ]:
results = []
for rounding_mode in ["floored", "ceiled"]:
    factory = StochasticProgramFactory(
        demand[[rounding_mode]].rename(columns={rounding_mode: "demand"}),
        distances,
        probabilities,
        node_df,
        ALL_VEHICLE_TYPES,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    
    # assign all weight to first scenario
    stochastic_program.weighting = {0: 1}

    # discard non-anticipativity constraints
    stochastic_program.non_anticipativity_disabled = True

    stochastic_program.create_model()
    stochastic_program.solve()

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
    )


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  7276.816666666652
solve finished in 2.34 seconds
No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  9190.243333333376
solve finished in 1.86 seconds


In [ ]:
results = pd.DataFrame.from_dict(results)
results

,status,objective,expected_profit,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,vehicle_types
0,Optimal,7276.816667,7276.816667,8015.0,1232.0,9247.0,4737.0,2569.0,"[kick_scooter, bicycle, car]"
1,Optimal,9190.243333,9190.243333,8803.0,1256.0,10059.0,3921.0,2845.0,"[kick_scooter, bicycle, car]"


In [ ]:
results_df.to_pickle(PATH_RESULTS_VALUE_STOCHASTIC)

## Value-At-Risk

In [ ]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)

factory.include_methods = [None]
factory.set_initial_allocation(real_fleet_size)

stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]

for beta in [0, 0.25, 0.5, 0.75]:
    stochastic_program.create_model(beta=beta, alpha=0.9)
    stochastic_program.solve()

    results.append(
        {
            **stochastic_program.get_summary(),
            "beta": beta,
        }
    )
    print("\n")


_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.01 seconds
_convert_demand finished in 0.03 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.04 seconds
_set_max_demand finished in 0.01 seconds
No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  23305.031111109853
solve finished in 907.58 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  22869.23500000004
solve finished in 2583.36 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  22461.441111111133
solve finished in 2889.62 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  22068.56333333317
solve finished in 545.03 seconds




In [ ]:
results_df = pd.DataFrame.from_dict(results)

In [ ]:
results_df.to_pickle(PATH_RESULTS_VALUE_STOCHASTIC)